# Where Should Microsoft Put Investment in New Film Projects?

## As a fledging studio 'Microsoft Pictures' must ensure profitability by investing in projects that:

###    - Center around highly rated, viewed, and desired genres and story arcs
###    - Attach names of directors and actors that create buzz and viewers
###    - 